In [5]:
#I want to put  csv file in form of separate 2d numpy arrays
#How?.....
#The arduino gets a reading of each sensor every 3 seconds.(Confirm with devs)
# there are 8 sensors per arduino(i think)
# so..8*3 = 24
# so every 24 seconds, separate csv
#grid (estimated is 6 by 10)
#current total: 

In [6]:
import numpy as np
import pandas as pd

In [7]:
Sensor_data_df = pd.read_csv("SensorData_trial.csv") # load csv file

In [8]:
Sensor_data_df # print df

,Location,Date,Time,Sensor Type,Reading,
0,I2,10/21/2021 12:00:00 AM,14:11:15,Water Level,237,
1,I2,10/21/2021 12:00:00 AM,14:11:35,Water Level,219,
2,I2,10/21/2021 12:00:00 AM,14:11:56,Water Level,225,
3,I2,10/21/2021 12:00:00 AM,14:12:17,Water Level,226,
4,I2,10/21/2021 12:00:00 AM,14:12:37,Water Level,220,
...,...,...,...,...,...,...
5315,F2,10/21/2021 12:00:00 AM,14:37:21,Water Level,27,
5316,F2,10/21/2021 12:00:00 AM,14:37:43,Water Level,28,
5317,F2,10/21/2021 12:00:00 AM,14:38:04,Water Level,27,
5318,F2,10/21/2021 12:00:00 AM,14:38:25,Water Level,28,


In [9]:
Sensor_ids = Sensor_data_df.Location.unique() # get unique values in Location(Sensor IDS)
print("Number of sensors: ",len(Sensor_ids))
print("Sensor array: ", Sensor_ids)

Number of sensors:  64
Sensor array:  ['I2' 'J2' 'G1' 'H1' 'I1' 'J1' 'G2' 'H2' 'D9' 'A10' 'B10' 'C10' 'D10' 'A9'
 'B9' 'C9' 'C1' 'D1' 'A2' 'B2' 'C2' 'D2' 'A1' 'B1' 'F8' 'F9' 'F10' 'E7'
 'E8' 'E9' 'E10' 'F7' 'A5' 'B5' 'C5' 'D5' 'A6' 'B6' 'C6' 'D6' 'B3' 'C3'
 'D3' 'A4' 'B4' 'C4' 'D4' 'A3' 'C8' 'D8' 'A7' 'B7' 'C7' 'D7' 'A8' 'B8'
 'F3' 'F4' 'E1' 'E2' 'E3' 'E4' 'F1' 'F2']


In [10]:
Start_row = Sensor_data_df.iloc[0]#get first time value in df
End_row = Sensor_data_df.iloc[-1] #get last time value in df
Start_time = Start_row[2] # Save start time obj
End_time = End_row[2] # Save end time obj

In [11]:
print("Start time for curr_csv: ", Start_time)
print("End time for curr_csv: ", End_time)

Start time for curr_csv:   14:11:15
End time for curr_csv:   14:38:46


In [12]:
#convert string to date-time and subtract end -start to get estimate of how many "frames" there should be
from datetime import datetime, date

In [13]:
#remove space in string first
Start_time = Start_time.replace(" ", "")
End_time = End_time.replace(" ", "")

In [14]:
time = datetime.strptime(Start_time, '%H:%M:%S').time() #start time converted to date time object
time1 = datetime.strptime(End_time, '%H:%M:%S').time()  #end time converted to date time object

In [15]:
datetime_obj = datetime.combine(date.today(), time1) - datetime.combine(date.today(), time)
print("Total time (hrs:mins:secs) for data to be generated: ",datetime_obj )

Total time (hrs:mins:secs) for data to be generated:  0:27:31


In [16]:
# 27 mins worth of data. im therefore expecting x frames from data set
# first step though is to see if i can split dataframe using date time

#current issue: datetime obj is in the form of timedelta while start and end is in the form of datatime
#we need to convert both into timedelta for the next step

In [17]:
def convert_datetime_to_timedelta(datetime_obj):
    return datetime.combine(date.min, datetime_obj) - datetime.min

In [18]:
if convert_datetime_to_timedelta(datetime.strptime("0:27:31", '%H:%M:%S').time()) == datetime_obj:
    print("True")#this funtion just tests if string obtained from df can be equated to a start-end

True


In [19]:
# step 1, convert each string in Time to a datetime object wih format '%H:%M:%S'
# issue: space - remove whitespace first(done)

In [20]:
#force update on pandas series
Time_data = Sensor_data_df[" Time"]# using df.time doest work because theres a white space for some reason
for x in range(len(Time_data)): 
    val_ = Time_data[x].replace(" ", "")#remove white space
    val_ = datetime.strptime(val_, '%H:%M:%S').time()#change to date time
    Time_data.update({x: val_})# update series

In [21]:
#series updated. next step, replace current updateed series with old one 
Time_data[0]

datetime.time(14, 11, 15)

In [22]:
Sensor_data_df[" Time"] = Time_data# update dataframe from obj to datetime

In [23]:
#sort dataframe first
Sensor_data_df = Sensor_data_df.sort_values(by=[" Time"])#sort by time. if you sort by any other value things will get weird
Sensor_data_df

,Location,Date,Time,Sensor Type,Reading,
0,I2,10/21/2021 12:00:00 AM,14:11:15,Water Level,237,
1524,C1,10/21/2021 12:00:00 AM,14:11:15,Water Level,4,
766,D9,10/21/2021 12:00:00 AM,14:11:15,Water Level,76,
2251,F8,10/21/2021 12:00:00 AM,14:11:16,Water Level,23,
3375,B3,10/21/2021 12:00:00 AM,14:11:16,Water Level,83,
...,...,...,...,...,...,...
95,I2,10/21/2021 12:00:00 AM,14:44:43,Water Level,225,
191,J2,10/21/2021 12:00:00 AM,14:44:46,Water Level,0,
287,G1,10/21/2021 12:00:00 AM,14:44:48,Water Level,10,
383,H1,10/21/2021 12:00:00 AM,14:44:51,Water Level,2,


In [24]:
#split dataframe according to "frames"
# according to devs(see first cell), split df every 24 secs
#we need to loop through others while keeping track of time passed.
#if enough time has passed(while appending rows), set new start point and continue process

In [25]:
Start = True #set start point
End = False
temp_array = []
temp = []
frame = 0
for index, row in Sensor_data_df.iterrows():
    if Start == True:
        temp_array.append(temp)
        temp = []
        start_time = row[" Time"]#get start time
        Start = False#set condition to false
    #every 24 seconds, set a new start time, if end - start = 24
    if datetime.combine(date.today(), row[" Time"]) - datetime.combine(date.today(), start_time) > convert_datetime_to_timedelta(datetime.strptime("0:00:24", '%H:%M:%S').time()):
        Start = True
        frame+=1
    temp.append(row)
    #print("Time passed: ", datetime.combine(date.today(), row[" Time"]) - datetime.combine(date.today(), start_time))
    #print(frame)

In [26]:
print("number of frames: ", len(temp_array))

number of frames:  79


In [27]:
#for i in temp_array:
    #print("Sensors per frame =", len(i))

In [28]:
item_array = []
for item in temp_array[1]:
    #print(item[0])
    item_array.append(item[0])

In [29]:
unique_items = set(item_array)
unique_items = list(unique_items)#convert type set to list - difficult to loop through set type for this scenario
unique_items.sort()#sort them
#unique_items

In [30]:
import re
>>> r = re.compile("([a-zA-Z]+)([0-9]+)")
>>> m = r.match(unique_items[1])
>>> m.group(1)
>>> m.group(2)

'10'

In [31]:
letter = []
ints = []
r = re.compile("([a-zA-Z]+)([0-9]+)")
for value in unique_items:
    g = r.match(value)
    letter.append(g.group(1))
    ints.append(int(g.group(2)))

In [32]:
zipped_lists = zip(letter, ints)
sorted_pairs = sorted(zipped_lists)

tuples = zip(*sorted_pairs)
list1, list2 = [ list(tuple) for tuple in  tuples]

In [33]:
for i in range(len(list1)):
    list1[i] = list1[i]+str(list2[i])

In [34]:
unique_items = list1

In [35]:
# I need to arrange all these into a single 2d array accounting for:
# duplicate values
# create blank 10 by 10 array first
empty_array = np.zeros((10, 10), int)

In [36]:
empty_array

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [37]:
def convert_to_255_range(empty_array):
    return ((empty_array - empty_array.min()) * (1/(empty_array.max() - empty_array.min()) * 255)).astype('uint8')

In [38]:
#loop through array filling values where necessary
convert_to_255_range(empty_array)

c:\users\scowt\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in long_scalars
  
c:\users\scowt\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in multiply
  


array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=uint8)

In [39]:
import collections
print([item for item, count in collections.Counter(item_array).items() if count > 1])

['I2', 'C1', 'D9', 'F8', 'B3', 'A5', 'F3', 'J2', 'A10', 'C8', 'F9', 'D1', 'B5', 'C3', 'D8']


In [40]:
#next step? arrange received data from each temp array into respective section in 2d array
#how? First- i need 2d empty array full of zeros to fill (done)
#data (done)

In [41]:
temp_array.pop(0)#careful with this should only be done once

[]

In [42]:
frame_data = []
for frame in range(len(temp_array)):
    #frame_data = []
    sensor_tracker = 0
    empty_array = np.zeros((10, 10), int)
    #frame = 70
    c = 0
    for row in range(len(empty_array[0])):
        for col in range(len(empty_array[1])):
            #access row col index
            #fill necessarily
            #row:
            #0 = A
            #1 = B......
            #col:
            #0 = 1
            #1 = 2....
            #eg in this instance, search for all cases of A1 in this frame
            #then if there are duplicates, either: average values or pick last
            #then store in 2d array
            #so...when searching for all cases of A1, get unique list str
            #loop/get curr index in unique string
            #loop through time frame data either for avg or last
            #update 2d array and move on to next
            if sensor_tracker < len(unique_items):
                #print("id: ",sensor_tracker)
                sensor_id = unique_items[sensor_tracker]
                c +=1
            for item in temp_array[frame]:
                #if item!=[]:
                #print(item[0][0])
                #Start_time = Start_time.replace(" ", "")
                if item[0].replace(" ","") == sensor_id.replace(" ",""):
                    #print(item[0])
                    #update array
                    empty_array[row,col] = int(item[4])
                    break
            sensor_tracker+=1
    frame_data.append(convert_to_255_range(empty_array))

In [43]:
len(frame_data)

78

In [44]:
frame_data_numpy = np.array(frame_data)
frame_data_numpy.shape

(78, 10, 10)

In [45]:
y = np.expand_dims(frame_data_numpy, axis=1)
y.shape

(78, 1, 10, 10)

In [46]:
len(y)

78

In [47]:
#save numpy data in folder as tensors
import torch
import os
import sklearn.preprocessing as scaler

In [52]:
y[1].shape

(1, 10, 10)

In [51]:
scaler.minmax_scale(y[1].squeeze(),feature_range=(0,1))

array([[1.        , 0.04313725, 0.92957746, 1.        , 0.        ,
        0.11842105, 1.        , 0.32      , 0.        , 0.57894737],
       [0.06122449, 0.38039216, 0.56338028, 0.20895522, 0.02197802,
        0.11842105, 0.        , 0.16      , 0.        , 0.68421053],
       [0.02040816, 0.08627451, 0.04225352, 0.19402985, 0.        ,
        0.        , 0.04878049, 0.        , 0.06451613, 0.05263158],
       [0.89795918, 1.        , 1.        , 0.28358209, 0.        ,
        1.        , 0.        , 0.01333333, 1.        , 1.        ],
       [0.        , 0.        , 0.        , 0.        , 0.62637363,
        0.01315789, 0.        , 1.        , 0.        , 0.        ],
       [0.        , 0.        , 0.91549296, 0.19402985, 1.        ,
        0.01315789, 0.        , 0.10666667, 0.        , 0.42105263],
       [0.08163265, 0.39215686, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.       

In [70]:
inputs_=[]
for data in range(len(y)):
    #print(y[data].shape)
    norm_ = scaler.minmax_scale(y[data].squeeze(),feature_range=(0,1))
    #print(norm_.shape)
    t = torch.from_numpy(norm_)
    t = t.unsqueeze(dim = 0)
    #print(t.size())
    #break
    str_= os.path.join('frame_tensors/' , str(data) + '.pt')
    inputs_.append(str(data)+'.pt')
    torch.save(t,str_ )
    #print(y[data].shape, type(t),t.size())

In [71]:
labels = y[1:]
labels.shape

(77, 1, 10, 10)

In [63]:
labels = np.concatenate((labels.squeeze(), y[-2]))
labels.shape

(78, 10, 10)

In [64]:
labels = np.expand_dims(labels, axis=1)
labels.shape

(78, 1, 10, 10)

In [73]:
labels_=[]
for data in range(len(labels)):
    norm_ = scaler.minmax_scale(labels[data].squeeze(),feature_range=(0,1))
    t = torch.from_numpy(norm_)
    t = t.unsqueeze(dim = 0)
    print(t.size())
    str_= os.path.join('frame_output_tensors/' , str(data) + '.pt')
    labels_.append(str(data)+'.pt')
    torch.save(t,str_ )

torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 1

In [60]:
labels.shape

(78, 1, 10, 10)

In [64]:
lstm_df = {
    'Inputs':inputs_,
    'Labels':labels_,
    }

lstm_df = pd.DataFrame(lstm_df, columns = ['Inputs','Labels'])
file_ = 'conv-lstm.csv'
lstm_df.to_csv(file_, encoding='utf-8')
lstm_df.head()

,Inputs,Labels
0,0.pt,0.pt
1,1.pt,1.pt
2,2.pt,2.pt
3,3.pt,3.pt
4,4.pt,4.pt


In [44]:
#transpose this way for gif purposes
np.transpose(y, (0, 2, 3, 1)).shape

(78, 10, 10, 1)

In [257]:
y = y.transpose(0,2,3,1)

In [258]:
y.shape

(77, 10, 10, 1)

In [262]:
from moviepy.editor import ImageSequenceClip
clip = ImageSequenceClip(list(y), fps=10)
clip.write_gif('test.gif', fps=10)

MoviePy - Building file test.gif with imageio.


In [1]:
frame_data_numpy[70]

NameError: name 'frame_data_numpy' is not defined

In [186]:
frames = np.random.randint(256, size=[78, 10, 10, 1], dtype=np.uint8)
frames.shape

(78, 10, 10, 1)

In [ ]:
#save all frames to files as tensors(to work with custom dataloader)
